In [21]:
import pathlib as pl
import csv
import pandas as pd
import findspark

In [22]:
# Assume that each data source location has its own .csv in the data_dir specified.
# Assume there is a subfolder named by the specified year.
# The column in each file for each datapoint's month is month_col.
# The column in each file for each datapoint's dependent data is data_col.
# The column name to use for station ids is id_col.
# The lat/lon of each station is in the file station_locs.
# Within that file, the station IDs are in column number station_ids.
# The longitude and latitude columns are lon_col, lat_col

data_dir = "extracted"
year = 2012
month_col = "Month"
data_col = "sm_depth_5cm"
id_col = "ID"
station_locs = "station_map.csv"
st_ids = 2
lon_col = "x"
lat_col = "y"
out_dir = "monthly_means"

In [23]:
folder = pl.Path(data_dir).joinpath(str(year))
sm = pd.DataFrame()
sm.name = str(year)
for file in folder.iterdir():
    st_id = file.stem
    try:
        int(st_id)
        st_data = pd.read_csv(file, delimiter=",", )
        st_data[id_col] = st_id
        sm = sm.append(st_data, ignore_index=True)
    except:
        print(f"File has non-integer name: {file}.")
sm.head()

File has non-integer name: extracted/2012/station_list.csv.


,Month,Day,DayOfYear,sm_depth_5cm,ID
0,1,1,1,0.20700,0251210112
1,1,2,2,0.21975,0251210112
2,1,3,3,0.22346,0251210112
3,1,4,4,0.22471,0251210112
4,1,5,5,0.22521,0251210112


In [24]:
sm_monthly = pd.DataFrame(sm.groupby([month_col, id_col])[data_col].mean())
sm_monthly.head()

sm_depth_5cm
Month ID                      
1     0001900212      0.347124
      0010019412      0.244877
      0010029412      0.250643
      0010039412      0.270259
      0010059498      0.362123

In [25]:
st_loc = pd.read_csv(station_locs, index_col=st_ids)
st_loc.head()

,x,y
id,,
1900212,-97.46,35.24
10019412,-98.02,34.80
10029412,-96.66,34.79
10039412,-99.34,34.59
10059498,-98.67,36.78


In [26]:
sm_m2 = pd.DataFrame(dtype="object")
for row in sm_monthly.itertuples():
    #print(row)
    m,i = row[0]
    xy = (st_loc.loc[int(i)])
    x = xy[lon_col]
    y = xy[lat_col]
    new_dict = {month_col: m, id_col: i, lon_col: x, lat_col: y, data_col: row[1]}
    new_row = pd.Series(new_dict, dtype="object")
    #print(new_row)
    sm_m2 = sm_m2.append(new_row, ignore_index=True)
sm_m2.head()

,ID,Month,sm_depth_5cm,x,y
0,0001900212,1.0,0.347124,-97.46,35.24
1,0010019412,1.0,0.244877,-98.02,34.80
2,0010029412,1.0,0.250643,-96.66,34.79
3,0010039412,1.0,0.270259,-99.34,34.59
4,0010059498,1.0,0.362123,-98.67,36.78


In [30]:
for i in range(1,13):
    monthly = sm_m2[sm_m2[month_col]==i]
    monthly = monthly[[lon_col, lat_col, data_col]].sort_values([lon_col, lat_col, data_col])#.reset_index(drop=True)
    print(f"The means for month {i} are:\n{monthly.head(2)}\n...")
    out_path = pl.Path(out_dir).joinpath(f"{year}_{i:02}.csv")
    monthly.to_csv(path_or_buf=out_path, index=False, header=False)

The means for month 1 are:
          x      y  sm_depth_5cm
339 -163.42  64.53      0.041263
133 -155.36  20.01      0.033414
...
The means for month 2 are:
           x      y  sm_depth_5cm
1042 -163.42  64.53      0.036214
833  -155.36  20.01      0.044167
...
The means for month 3 are:
           x      y  sm_depth_5cm
1535 -155.36  20.01      0.131000
1536 -155.35  19.55      0.165452
...
The means for month 4 are:
           x      y  sm_depth_5cm
2355 -163.42  64.53      0.048036
2240 -155.36  20.01      0.132200
...
The means for month 5 are:
           x      y  sm_depth_5cm
2978 -163.42  64.53      0.106143
2864 -155.36  20.01      0.136065
...
The means for month 6 are:
           x      y  sm_depth_5cm
3497 -163.42  64.53      0.411883
3507 -150.78  62.63      0.144618
...
The means for month 7 are:
           x      y  sm_depth_5cm
3976 -163.42  64.53      0.467417
3986 -150.78  62.63      0.120875
...
The means for month 8 are:
           x      y  sm_depth_5cm
4451 -163.4